In [ ]:
%matplotlib inline
#%connect_info
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt

def plot_3d(points):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1, projection='3d')
    surf = ax.scatter(points[:,0],points[:,1],points[:,2], c = (np.arange(points.shape[0])[:]))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

uneven_base_n takes an array of dimension sizes at construction, calculate then returns the value given as a number calculated in that Base

In [ ]:
class UnevenBase(object):
    def __init__(self, base_distrib):
        self.bd = base_distrib
    def calculate(self, value):
        coords = np.zeros(len(self.bd))
        for d in range(len(self.bd)):
            coords[d] = (value / np.prod(self.bd[:d])) % self.bd[d]
        return coords
    def c(self, value_array):
        coords = np.zeros((value_array.shape[0], self.bd.shape[0]))
        for d in range(self.bd.shape[0]):
            coords[:,d] = (value_array[:] / np.prod(self.bd[:d])) % self.bd[d]
        return coords
    def d(self, coords):
        vals = np.zeros(coords.shape[0])
        for d in range(self.bd.shape[0]):
            vals[:] += coords[:, d] * np.prod(self.bd[:d])
        return np.int_(vals)

In [ ]:
b = np.asarray([7,5,3])
t = UnevenBase(b)
bb = np.asarray([5,5,3])
tt = UnevenBase(bb)
bbb = np.asarray([3,3,3])
ttt = UnevenBase(bbb)
bbbb = np.asarray([1,1,1])
tttt = UnevenBase(bbbb)

d = np.arange(np.prod(b))
l = t.c(d) - (b / 2)

dd = np.arange(np.prod(bb))
ll = tt.c(dd) - (bb / 2)

ddd = np.arange(np.prod(bbb))
lll = ttt.c(ddd) - (bbb / 2)

dddd = np.arange(np.prod(bbbb))
llll = tttt.c(dddd) - (bbbb / 2)

let's stop worrying about the intersection, in fact, every value we're concerned with is available in the set: d - we can guarantee that `d/2 == the center` of the coordinate space - **!!REGARDLESS!!** of the dimensions sizes as long as, for each dimension, `size % 2 != 0`

if we examine the set of coordinates we generate by successively examining coordinate spaces beginning with a singular space (i.e. `dimensions=[1,1,1...,1]`) and increase the space stepwise with `dimensions[i] = min(lim_dimensions[i], dimensions[i]+2)` until `dimensions[:] == lim_dimensions[:]` - we can take the coordinates generated by *UnevenBase* counting - which essentially flattens the numbers we're counting into the cartesian plane regardless of the base we're counting them into, meaning they're being mapped into a space that let's us reduce them into any other base for as many dimensions as they contain information for in the cartesian space.

In [ ]:
aaaa = t.d(llll)

mmmm = (t.calculate(np.prod(b) / 2 + aaaa) - (b / 2))[:, None].T

plot_3d(mmmm)

In [ ]:
# 'a' here represents the indices relative to d / 2 we would need to visit in l 
# i.e. t.c((np.prod(b)/2) + a[:]) for all indices of a
aaa = t.d(lll)

qqq = np.setdiff1d(aaa, aaaa)

mmm = t.c(np.prod(b) / 2 + aaa) - (b / 2)
zzz = t.c(np.prod(b) / 2 + qqq) - (b / 2)

plot_3d(mmm)
plot_3d(zzz)

In [ ]:
aa = t.d(ll)

qq = np.setdiff1d(aa, aaa) 

mm = t.c(np.prod(b) / 2 + aa) - (b / 2)
zz = t.c(np.prod(b) / 2 + qq) - (b / 2)

plot_3d(mm)
plot_3d(zz)

In [ ]:
a = t.d(l)

q = np.setdiff1d(a, aa)

m = t.c(np.prod(b) / 2 + a) - (b / 2)
z = t.c(np.prod(b) / 2 + q) - (b / 2)

plot_3d(m)
plot_3d(z)

wow! look at that!  a perfect expansion in uneven space!

we can take from the following from teh above experiment:
flattening the coordinates in and out of cartesian space is an unnecessary step!
instead, we need to focus on elimating the following expansion (per notation above):
```
given set 'a' (existing points in an integer plane with dimensions [7,5,3]) and 
'aa' (existing points in an integer plane with dimensions [5,5,3]:
q = a \ b
```
( where \ denotes [[complement][https://en.wikipedia.org/wiki/Complement_(set_theory)]] )

if we can avoid taking the complement, and instead calculate a cartesian space whcih manifolds on that complementary space -- we're golden!

along those lines - here's known data we can pass into our counting model:
   * size of current regular space (i.e. [5,5,3])
   * size of last regular space (i.e. [3,3,3])
   * size of spatial set for each space (i.e. ∏[5,5,3])
   
the trick is trying to discern the manifold space - which appears in our example model as a toroidal plane, as disjunct n-1 planes, and as a spherical plane in different expansion scenarios

now - the loser's resolution would be to to run through every point in the current regular space `O(∏[X,Y,Z])` - testing that the points we're calculating are not present in the last-regular-space - and YET! - when we follow the stepping mechanism referred to above, this would inflate the number of points we'd examine from 
`O(∏[X,Y,Z])` to `O(∏[X,Y,Z] - ∏[min(X-2,1),min(Y-2,1),min(Z-2,1)]...-∏[1,1,1])` YIKES! 

In [ ]:
class ManifoldCount(object):
    def __init__(self, d, cd):
        self.d = d
        self.ds = np.prod(self.d)
        self.cd = cd
        self.cs = np.prod(self.cd)
        self.ms = self.ds - self.cs

In [ ]:
#moments of power:
jkl = np.ediff1d(q)
klj = np.ediff1d(qq)
ljk = np.ediff1d(qqq)

qwe = np.prod(b) / 2 + q
weq = np.prod(b) / 2 + qq
eqw = np.prod(b) / 2 + qqq


asd =  t.c(qwe) - (b / 2)
sda =  t.c(weq) - (b / 2)
das =  t.c(eqw) - (b / 2)

print jkl
print klj
print ljk

In [ ]:
def plot_1d(points,color,offset,size):
    plt.plot(points, np.zeros_like(points)+offset, '.', markersize=size, c=color)

In [ ]:
size = 12.0
plot_1d(qwe, 'blue', 0, size)
plot_1d(weq, 'green', 0, size)
plot_1d(eqw, 'red', 0, size)

plt.plot(np.prod([7,5,3]) / 2, 0 ,'d', markersize=size, c='black')

plt.xlim([0 - 3, 3 + np.prod([7,5,3])])

fig_size = plt.rcParams["figure.figsize"]

fig_size[0] = 12
fig_size[1] = 1
plt.rcParams["figure.figsize"] = fig_size
plt.gca().axes.get_yaxis().set_visible(False)

plt.show()

size = 90.0
plot_1d(qwe, 'blue', 0,size)
plot_1d(weq, 'green', 0, size)
plot_1d(eqw, 'red', 0, size)

plt.plot(np.prod([7,5,3]) / 2, 0 ,'d', markersize=size, c='black')

plt.xlim([52 - 10, 52 + 10])
plt.show()

In [ ]:
size = 12.0
plot_1d(qwe, 'blue', 3,size)
plot_1d(weq, 'green', 2, size)
plot_1d(eqw, 'red', 1, size)

plt.plot(np.prod([7,5,3]) / 2, 0 ,'d', markersize=size, c='black')

plt.xlim([0 - 3, 3 + np.prod([7,5,3])])
plt.ylim([-1,4])
plt.show()

In [ ]:
def max_(a,b):
    test = np.where(a > b)
    a[test] = b[test]
    return a

m_dim = np.asarray([7,5,3])
dim = np.asarray([1,1,1])
for i in np.arange(np.prod(m_dim)):
    t = np.prod(dim)
    if i % t == 0:
        dim = max_(dim+2,m_dim)
    print dim
        
        